In [1]:
import pandas as pd
import numpy as np 
import pylab as P 
from yahoo_finance import Share
from pprint import pprint
# Import the module for plotting
import matplotlib.pyplot as plt
from pandas import Series,DataFrame,merge,concat
import pylab as P 
from datetime import datetime
from pandas.tseries.offsets import Day, BusinessDay
%matplotlib inline

# Finance Data

In [2]:
googleS = Share('GOOGL')
google_hist = googleS.get_historical('2013-01-01', '2016-01-01')
googleF = DataFrame(google_hist)
googleF['Date'] = googleF['Date'].apply(lambda x: datetime.strptime(str(x),'%Y-%m-%d'))
googleF.index = googleF['Date']
googleF.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 756 entries, 2015-12-31 to 2013-01-02
Data columns (total 8 columns):
Adj_Close    756 non-null object
Close        756 non-null object
Date         756 non-null datetime64[ns]
High         756 non-null object
Low          756 non-null object
Open         756 non-null object
Symbol       756 non-null object
Volume       756 non-null object
dtypes: datetime64[ns](1), object(7)
memory usage: 53.2+ KB


In [3]:
googleF.tail()

,Adj_Close,Close,Date,High,Low,Open,Symbol,Volume
Date,,,,,,,,
2013-01-08,367.017034,733.300033,2013-01-08,736.300021,724.430006,735.540039,GOOGL,3348800
2013-01-07,367.742742,734.749998,2013-01-07,739.380007,730.580038,735.449981,GOOGL,3308000
2013-01-04,369.354344,737.96998,2013-01-04,741.470017,727.679987,729.340008,GOOGL,5521400
2013-01-03,362.197209,723.670024,2013-01-03,731.930005,720.720035,724.929993,GOOGL,4631700
2013-01-02,361.987004,723.250034,2013-01-02,727.000004,716.550015,719.420006,GOOGL,5077500


# Tweets Data

In [5]:
# data import using the Data combined by Sectors
rawdata = pd.read_csv('../data/preprocessed/DataBySectors.csv')
data = rawdata.ix[:,3:]

In [6]:
# select company data
googleT = data[data['SYMBOL'] == 'GOOG']
print(googleT['EXCHANGE'].unique())
print(googleT.count().unique())
print(googleT['BUSINESS_SECTOR'].unique())
googleT.head()

['NASDAQ']
[2289]
['Technology']


,SYMBOL,TIMESTAMP_UTC,BULLISH_INTENSITY,BEARISH_INTENSITY,BULL_MINUS_BEAR,BULL_SCORED_MESSAGES,BEAR_SCORED_MESSAGES,BULL_BEAR_MSG_RATIO,TOTAL_SCANNED_MESSAGES,TITLE,EXCHANGE,BUSINESS_SECTOR
5289172,GOOG,2009-07-10T04:00:00Z,2.3,0,2.3,1,0,0,2,Google Inc.,NASDAQ,Technology
5289173,GOOG,2009-07-11T04:00:00Z,0.0,0,0.0,0,0,0,2,Google Inc.,NASDAQ,Technology
5289174,GOOG,2009-08-21T04:00:00Z,0.0,0,0.0,0,0,0,1,Google Inc.,NASDAQ,Technology
5289175,GOOG,2009-08-23T04:00:00Z,0.0,0,0.0,0,0,0,1,Google Inc.,NASDAQ,Technology
5289176,GOOG,2009-08-31T04:00:00Z,0.0,0,0.0,0,0,0,1,Google Inc.,NASDAQ,Technology


In [7]:
# change timestamp for table merge
googleT['TIMESTAMP_UTC'] = googleT['TIMESTAMP_UTC'].apply(lambda x: datetime.strptime(x,'%Y-%m-%dT%H:%M:%SZ'))
googleT['Date'] = googleT['TIMESTAMP_UTC'].apply(lambda x: x.strftime('%x'))
googleT['Date'] = googleT['Date'].apply(lambda x: pd.to_datetime(x))
googleT.info()

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2289 entries, 5289172 to 5291460
Data columns (total 13 columns):
SYMBOL                    2289 non-null object
TIMESTAMP_UTC             2289 non-null datetime64[ns]
BULLISH_INTENSITY         2289 non-null float64
BEARISH_INTENSITY         2289 non-null float64
BULL_MINUS_BEAR           2289 non-null float64
BULL_SCORED_MESSAGES      2289 non-null float64
BEAR_SCORED_MESSAGES      2289 non-null float64
BULL_BEAR_MSG_RATIO       2289 non-null float64
TOTAL_SCANNED_MESSAGES    2289 non-null float64
TITLE                     2289 non-null object
EXCHANGE                  2289 non-null object
BUSINESS_SECTOR           2289 non-null object
Date                      2289 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(7), object(4)
memory usage: 250.4+ KB


# Merge Finance and Tweets Data

In [8]:
# merge
google = merge(googleT,googleF,on='Date',how = "right")

In [9]:
google.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 756 entries, 0 to 755
Data columns (total 20 columns):
SYMBOL                    756 non-null object
TIMESTAMP_UTC             756 non-null datetime64[ns]
BULLISH_INTENSITY         756 non-null float64
BEARISH_INTENSITY         756 non-null float64
BULL_MINUS_BEAR           756 non-null float64
BULL_SCORED_MESSAGES      756 non-null float64
BEAR_SCORED_MESSAGES      756 non-null float64
BULL_BEAR_MSG_RATIO       756 non-null float64
TOTAL_SCANNED_MESSAGES    756 non-null float64
TITLE                     756 non-null object
EXCHANGE                  756 non-null object
BUSINESS_SECTOR           756 non-null object
Date                      756 non-null datetime64[ns]
Adj_Close                 756 non-null object
Close                     756 non-null object
High                      756 non-null object
Low                       756 non-null object
Open                      756 non-null object
Symbol                    756 non-null objec

# Dataset Preprocessing

In [10]:
def volume_dataset(df):
    df.Volume = df.Volume.astype(float)
    df.Adj_Close = df.Adj_Close.astype(float)
    return df.ix[:,[12,2,3,4,5,6,7,13,15,16,17,19,10,11,0]]

def log_return(df):
    return df.Adj_Close.pct_change()

def log_bulliness(df):
    f = DataFrame(df.ix[:,['BULL_SCORED_MESSAGES','BEAR_SCORED_MESSAGES']])
    f['posVol'] = 1 + f['BULL_SCORED_MESSAGES']
    f['negVol'] = 1 + f['BEAR_SCORED_MESSAGES']
    return np.log(f.posVol)-np.log(f.negVol)

def log_ratio(df):
    f = DataFrame(df.ix[:,['BULL_SCORED_MESSAGES','BEAR_SCORED_MESSAGES']])
    return np.log(f.BULL_SCORED_MESSAGES)-np.log(f.BEAR_SCORED_MESSAGES)

def TISf(df):
    f = DataFrame(df.ix[:,['BULL_SCORED_MESSAGES','BEAR_SCORED_MESSAGES']])
    return (1+f['BULL_SCORED_MESSAGES'])/(1+ f['BULL_SCORED_MESSAGES']+f['BEAR_SCORED_MESSAGES'])

def RTISf(df):
    return df.TIS.pct_change()

In [11]:
# Message Volumns 
google.index = google['Date']
googleV = volume_dataset(google)
googleV['Bullishness'] = log_bulliness(googleV)
googleV['BuBeRatio'] = log_ratio(googleV)
googleV['TIS'] = TISf(googleV)
googleV['RTIS'] = RTISf(googleV)
googleV['Return'] = log_return(googleV)
googleV.head()

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



,Date,BULLISH_INTENSITY,BEARISH_INTENSITY,BULL_MINUS_BEAR,BULL_SCORED_MESSAGES,BEAR_SCORED_MESSAGES,BULL_BEAR_MSG_RATIO,Adj_Close,High,Low,Open,Volume,EXCHANGE,BUSINESS_SECTOR,SYMBOL,Bullishness,BuBeRatio,TIS,RTIS,Return
Date,,,,,,,,,,,,,,,,,,,,
2013-01-02,2013-01-02,1.95,1.64,0.31,65,38,1.71,361.987004,727.000004,716.550015,719.420006,5077500,NASDAQ,Technology,GOOG,0.526093,0.536801,0.634615,NaN,NaN
2013-01-03,2013-01-03,1.72,1.77,-0.05,216,84,2.57,362.197209,731.930005,720.720035,724.929993,4631700,NASDAQ,Technology,GOOG,0.937246,0.944462,0.720930,0.136011,0.000581
2013-01-04,2013-01-04,1.84,1.63,0.21,229,96,2.39,369.354344,741.470017,727.679987,729.340008,5521400,NASDAQ,Technology,GOOG,0.863368,0.869374,0.705521,-0.021373,0.019760
2013-01-07,2013-01-07,1.79,1.42,0.37,105,67,1.57,367.742742,739.380007,730.580038,735.449981,3308000,NASDAQ,Technology,GOOG,0.443931,0.449268,0.612717,-0.131541,-0.004363
2013-01-08,2013-01-08,1.64,1.58,0.06,96,67,1.43,367.017034,736.300021,724.430006,735.540039,3348800,NASDAQ,Technology,GOOG,0.355203,0.359656,0.591463,-0.034687,-0.001973


# Correlation Analysis

- stock return with sentiment strength and message volume
- stock trading volume with sentiment strength and message volume

In [12]:
i = 0
resCorr = DataFrame(index = ['BULLISH_INTENSITY','BEARISH_INTENSITY','BULL_MINUS_BEAR','BULL_SCORED_MESSAGES',
       'BEAR_SCORED_MESSAGES','Bullishness','TIS','RTIS'] ,columns = ['Return Correlation','Volume Correlation'])
for column in resCorr.index:
    resCorr.ix[i,0] = googleV.Return.corr(googleV[column])
    resCorr.ix[i,1] = googleV.Volume.corr(googleV[column])
    i += 1
resCorr

,Return Correlation,Volume Correlation
BULLISH_INTENSITY,0.148921,-0.080251
BEARISH_INTENSITY,-0.156589,-0.149391
BULL_MINUS_BEAR,0.220866,0.0710794
BULL_SCORED_MESSAGES,0.122681,0.105288
BEAR_SCORED_MESSAGES,0.0903416,0.632048
Bullishness,0.232106,-0.13226
TIS,0.244237,-0.144958
RTIS,0.0529734,-0.0458358


# Mutual Information Method

In [13]:
def calc_MI(X,Y,bins):

   c_XY = np.histogram2d(X,Y,bins)[0]
   c_X = np.histogram(X,bins)[0]
   c_Y = np.histogram(Y,bins)[0]

   H_X = shan_entropy(c_X)
   H_Y = shan_entropy(c_Y)
   H_XY = shan_entropy(c_XY)

   MI = H_X + H_Y - H_XY
   return MI

def shan_entropy(c):
    c_normalized = c / float(np.sum(c))
    c_normalized = c_normalized[np.nonzero(c_normalized)]
    H = -sum(c_normalized* np.log2(c_normalized))  
    return H

def increaseMI(non,s):
    return ((s-non)/non)*100

# Test set
# A = np.array([[ 2.0,  140.0,  128.23, -150.5, -5.4  ],
#               [ 2.4,  153.11, 130.34, -130.1, -9.5  ],
#               [ 1.2,  156.9,  120.11, -110.45,-1.12 ]])

# bins = 6 
# n = A.shape[1]
# matMI = np.zeros((n, n))

# for ix in np.arange(n):
#     for jx in np.arange(ix+1,n):
#         matMI[ix,jx] = calc_MI(A[:,ix], A[:,jx], bins)
        
# matMI

# another way calculate MI
from sklearn.metrics import mutual_info_score

def calc_MIa(x, y, bins):
    c_xy = np.histogram2d(x, y, bins)[0]
    mi = mutual_info_score(None, None, contingency=c_xy)
    return mi

# Surplus Information

In [14]:
def calc_suplsMI(df):
    i = 0
    suplsMI = DataFrame(index = ['BULLISH_INTENSITY','BEARISH_INTENSITY','BULL_MINUS_BEAR','BULL_SCORED_MESSAGES',
       'BEAR_SCORED_MESSAGES','Bullishness','TIS','RTIS'] ,columns = ['Return Surplus','Volume Surplus'])
    shift1_r = df.Return.shift(1)
    shift1_v = df.Volume.shift(1)
    for column in suplsMI.index:
#       MI for finance return and social data
        nonshift_r = calc_MI(df[column][1:],df.Return[1:],756)
        shifted_r = calc_MI(df[column][2:],shift1_r[2:],756)
#       MI for finance volume and social data
        nonshift_v = calc_MI(df[column][1:],df.Volume[1:],756)
        shifted_v = calc_MI(df[column][1:],shift1_v[1:],756)
#       surplus information for return and volume
        suplsMI.ix[i,0] = increaseMI(nonshift_r,shifted_r)
        suplsMI.ix[i,1] = increaseMI(nonshift_v,shifted_v)
        i += 1
    return suplsMI

In [15]:
calc_suplsMI(googleV)

,Return Surplus,Volume Surplus
BULLISH_INTENSITY,0.36794,0.210321
BEARISH_INTENSITY,-0.253979,0.60288
BULL_MINUS_BEAR,0.114736,0.207715
BULL_SCORED_MESSAGES,1.90443,-3.56775
BEAR_SCORED_MESSAGES,-0.379987,0.351438
Bullishness,0.0217993,-0.138218
TIS,0.118223,0.154559
RTIS,-0.282256,0.60654


In [16]:
nonshift = calc_MI(googleV.ix[1:,3],googleV.Return[1:],500)
shift1 = googleV.Return.shift(1)
shifted = calc_MI(googleV.ix[2:,3],shift1[2:],500)
print(nonshift, shifted)
incres = increaseMI(nonshift,shifted)
incres

(4.1653926756831279, 4.1521271418714853)


-0.31847018623440992

In [20]:
nonshift_ = calc_MIa(googleV.ix[1:,3],googleV.Return[1:],500)
shift1_ = googleV.Return.shift(1)
shifted_ = calc_MIa(googleV.ix[2:,3],shift1[2:],500)
print(nonshift_, shifted_)
increaseMI(nonshift_,shifted_)

(2.8902094481190925, 2.8810184320308512)


-0.31800519143077116

In [ ]:
data.EXCHANGE.unique()

In [ ]:
# googleV.ix[:,['Bullishness','Ratio']].plot()
googleV.Adj_Close.plot()
# rollMean = pd.rolling_mean(googleV.Adj_Close, 10)
# rollMean.plot()
rollStd = pd.rolling_std(googleV.Adj_Close, 5)
rollStd.plot()

In [ ]:
googleV.Adj_Close['2014'].plot()

In [ ]:
data.SYMBOL.unique()